In [9]:
import sys
sys.path.append('../..')
import numpy as np
# PyTorch
import torch
import torch.nn.functional as F
from torch import cuda
from torch.autograd import Variable
from torchvision.models.mobilenet import mobilenet_v2

# DLBio and own scripts
from DLBio.pytorch_helpers import get_device
import transforms_data as td
from helpers import cuda_conv
import metrics as M
import acquire_ear_dataset as a

from PIL import Image
import glob

In [10]:
class Config():
    DEVICE = get_device()
    DATASET_DIR = '../dataset/'
    AUTH_DATASET_DIR = '../auth_dataset/unknown-auth/'
    MODEL_DIR = './models/model_MN_color.pt'
    RESIZE_SMALL = False
    DATABASE_FOLDER = './embeddings/'

In [11]:
model = torch.load(Config.MODEL_DIR, map_location=torch.device(Config.DEVICE))
transformation = td.transforms_siamese_verification( td.get_resize(Config.RESIZE_SMALL) )
model.eval()

NameError: name 'mode' is not defined

In [12]:
def pipeline(input_, preprocess):
    #input_ = input_.convert("L")
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, td.get_resize(Config.RESIZE_SMALL)[0], td.get_resize(Config.RESIZE_SMALL)[1], 1)
    input_ = input_.permute(3, 0, 1, 2)

    if cuda.is_available():
        return input_.type('torch.cuda.FloatTensor')
    else:
        return input_.type('torch.FloatTensor')

In [17]:
# img = Image.open(Config.DATASET_DIR + 'janna_qua/janna_qua003.png')
# new_embedding = [model(Variable(pipeline(img,transformation))).cpu()]
# test = np.array(new_embedding)

# np.save('test.npy', test)

In [18]:
# loaded_test = np.load('test.npy', allow_pickle=True)

# print(loaded_test)
# print(new_embedding)

# value = F.pairwise_distance(loaded_test[0],new_embedding[0]).item()
# print(value)

[tensor([[0.4768, 0.4880, 0.5734,  ..., 0.4703, 0.4180, 0.4802]],
       requires_grad=True)]
[tensor([[0.4768, 0.4880, 0.5734,  ..., 0.4703, 0.4180, 0.4802]],
       grad_fn=<CopyBackwards>)]
3.577686220523901e-05


In [15]:
result_value = []
result_label = []

img = Image.open(Config.DATASET_DIR + 'janna_qua/janna_qua003.png')
new_embedding = model(Variable(pipeline(img,transformation))).cpu()

for label in os.listdir(Config.DATABASE_FOLDER):
    loaded_embedding = np.load(Config.DATABASE_FOLDER+label, allow_pickle=True)
    tmp = []    
    for embedding in loaded_embedding:
        dis = F.pairwise_distance(embedding,new_embedding)
        tmp.append(dis.item())
    result_value.append((min(tmp)))
    result_label.append(label)


In [16]:
result_value, result_label = zip(*sorted(zip(result_value, result_label)))
result_value = result_value[:10]
result_label = result_label[:10]

for idx, val in enumerate(result_label):
    print(str(idx+1) + ' : ' + ' ' + val + ' : ' + ' ' + str(result_value[idx]))

1 :  janna_qua.npy :  3.571590059436858e-05
2 :  tim_moe.npy :  0.09579116106033325
3 :  clara_pau.npy :  0.09974295645952225
4 :  collin_sch.npy :  0.11074963957071304
5 :  marina_fri.npy :  0.1114228144288063
6 :  clemens_blu.npy :  0.11395379155874252
7 :  beatrix_mah.npy :  0.1154462918639183
8 :  gregor_spi.npy :  0.11710232496261597
9 :  mohammed_muh.npy :  0.11872290819883347
10 :  konrad_von.npy :  0.11907429248094559
